# Model Training

## Requirements

In [1]:
## Load requirements
import pandas as pd
import plotly
import gc
import pickle
import matplotlib.pyplot as plt 

from typing import Tuple
from datetime import timedelta, datetime

from statsmodels.tsa.arima.model import ARIMA
import statsmodels.api as sm
import pmdarima as pm

from utils import load, slicer

from sklearn.metrics import r2_score

## Data Load

In [2]:
df, resample_df = load()

## Model Training

### ARIMA (Auto ARIMA)

In [ ]:
## iterations over stock name in DF
for stock in df["Name"].unique():
    print("Initializing processess for: ", stock)
    #initialize model
    try: 
        model = pm.auto_arima(resample_df[(resample_df['Name']==stock)].drop(columns=["Name"])['Close'], 
                            m=12, seasonal=True,
                        start_p=0, start_q=0, max_order=4, test='adf',error_action='ignore',  
                            suppress_warnings=True,
                        stepwise=True, trace=True)
    except:
        print("Error in model initialization: ", stock)
        pass

    #Splitting
    try:
        df_train, df_val, df_test = slicer(resample_df[(resample_df['Name']==stock)])
    except:
        print("Error in splitting: ",stock)
        pass

    #model training & fitting
    try: 
        model.fit(df_train['Close'])
    except:
        print("Error in model fitting: ",stock)
        pass
    
    #export model to .pkl (pickle.dump)
    try: 
        file_name = (f'../stock_forecasting/AutoARIMA_Models/PKL/{stock}_autoARIMA_')
        # save
        pickle.dump(model, open(file_name + 'Model.pkl', "wb"))
    except:
        print("Error in model export: ",stock)
        pass

    gc.collect() #RAM usage optimization